In [1]:
# import warnings
# warnings.filterwarnings('ignore')

In [2]:
# Initial imports
import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from path import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import precision_recall_curve
from sklearn.linear_model import LogisticRegression



In [3]:
#  Import and read the csv file(s)
#file_path = "../Mod20_Group_Challenge/iris.csv"
wine_df=pd.read_csv("https://raw.githubusercontent.com/saraegregg/Mod20_Group_Challenge/main/data_cleaning/cleaned_wine_data.csv")
wine_df.head(10)

,Unnamed: 0,country,description,points,price,province,taster_name,title,variety,winery
0,1,Portugal,"This is ripe and fruity, a wine that is smooth...",87,15.0,Douro,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
1,2,US,"Tart and snappy, the flavors of lime flesh and...",87,14.0,Oregon,Paul Gregutt,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
2,3,US,"Pineapple rind, lemon pith and orange blossom ...",87,13.0,Michigan,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
3,4,US,"Much like the regular bottling from 2012, this...",87,65.0,Oregon,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
4,5,Spain,Blackberry and raspberry aromas show a typical...,87,15.0,Northern Spain,Michael Schachner,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem
5,6,Italy,"Here's a bright, informal red that opens with ...",87,16.0,Sicily & Sardinia,Kerin O'Keefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo
6,7,France,This dry and restrained wine offers spice in p...,87,24.0,Alsace,Roger Voss,Trimbach 2012 Gewurztraminer (Alsace),Gewurztraminer,Trimbach
7,8,Germany,Savory dried thyme notes accent sunnier flavor...,87,12.0,Rheinhessen,Anna Lee C. Iijima,Heinz Eifel 2013 Shine Gewurztraminer (Rheinhe...,Gewurztraminer,Heinz Eifel
8,9,France,This has great depth of flavor with its fresh ...,87,27.0,Alsace,Roger Voss,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam
9,10,US,"Soft, supple plum envelopes an oaky structure ...",87,19.0,California,Virginie Boone,Kirkland Signature 2011 Mountain Cuvee Caberne...,Cabernet Sauvignon,Kirkland Signature


In [4]:
# Pull in top keywords
keywords = ['ripe', 'crisp', 'bright', 'dry', 'full', 'sweet', 'fresh', 'earthy', 'bitter', 'aftertaste']
for k in keywords:
    wine_df[k] = wine_df.description.apply(lambda x : 1 if x.find(k)>-1 else 0)
wine_df.head()

,Unnamed: 0,country,description,points,price,province,taster_name,title,variety,winery,ripe,crisp,bright,dry,full,sweet,fresh,earthy,bitter,aftertaste
0,1,Portugal,"This is ripe and fruity, a wine that is smooth...",87,15.0,Douro,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,1,0,0,0,0,0,1,0,0,0
1,2,US,"Tart and snappy, the flavors of lime flesh and...",87,14.0,Oregon,Paul Gregutt,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,0,1,0,0,0,0,0,0,0,0
2,3,US,"Pineapple rind, lemon pith and orange blossom ...",87,13.0,Michigan,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,0,0,0,1,0,0,0,0,0,0
3,4,US,"Much like the regular bottling from 2012, this...",87,65.0,Oregon,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,0,0,0,0,0,0,0,1,0,0
4,5,Spain,Blackberry and raspberry aromas show a typical...,87,15.0,Northern Spain,Michael Schachner,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem,0,0,0,0,1,0,1,0,0,0


In [5]:
def getPoints(points):
    if(points <= 85):
        return 'bad'
    elif(points<=90 ):
        return 'ok'
    elif(points<=95):
        return 'good'
    elif(points<=100):
        return 'great'
    else:
        return 'If this gets hit, we did something wrong!'

In [6]:
wine_df['Points'] = wine_df['points'].apply(getPoints)

In [7]:
country_counts=wine_df.country.value_counts()
country_counts

US                        37511
France                    17525
Italy                     10121
Spain                      6509
Portugal                   4870
Chile                      4305
Argentina                  3753
Austria                    2791
Germany                    2093
Australia                  2007
New Zealand                1270
South Africa               1220
Israel                      484
Greece                      461
Canada                      253
Hungary                     144
Bulgaria                    141
Romania                     120
Uruguay                     109
Turkey                       90
Georgia                      84
Slovenia                     80
Croatia                      71
England                      69
Mexico                       65
Moldova                      59
Brazil                       47
Lebanon                      35
Morocco                      28
Peru                         16
Ukraine                      14
Serbia  

In [8]:
# Determine which values to replace if counts are less than ..?
replace_country = list(country_counts[country_counts < 2000].index)

# Replace in dataframe
for coun in replace_country:
    wine_df.country= wine_df.country.replace(coun,"Other")
    
# Check to make sure binning was successful
wine_df.country.value_counts()

US           37511
France       17525
Italy        10121
Spain         6509
Other         4935
Portugal      4870
Chile         4305
Argentina     3753
Austria       2791
Germany       2093
Australia     2007
Name: country, dtype: int64

In [9]:
variety_counts=wine_df.variety.value_counts()
variety_counts

Pinot Noir                  9833
Chardonnay                  8292
Red Blend                   6842
Cabernet Sauvignon          6198
Bordeaux-style Red Blend    4816
                            ... 
Manzoni                        1
Syrah-Grenache-Viognier        1
Fruburgunder                   1
Chardonnay-Riesling            1
Bobal-Cabernet Sauvignon       1
Name: variety, Length: 654, dtype: int64

In [10]:
# Determine which values to replace if counts are less than ..?
replace_variety = list(variety_counts[variety_counts < 2000].index)

# Replace in dataframe
for var in replace_variety:
    wine_df.variety= wine_df.variety.replace(var,"Other")
    
# Check to make sure binning was successful
wine_df.variety.value_counts()

Other                       38956
Pinot Noir                   9833
Chardonnay                   8292
Red Blend                    6842
Cabernet Sauvignon           6198
Bordeaux-style Red Blend     4816
Riesling                     4722
Sauvignon Blanc              3838
Syrah                        3103
Rose                         3032
Malbec                       2524
Portuguese Red               2196
Merlot                       2068
Name: variety, dtype: int64

In [11]:
wine_df = wine_df.drop(columns=['description', 'province', 'title', 'winery', 'taster_name'], axis=1) 

In [12]:
# # hot encoding for country and taster name as they are limited categories. 
wine_df = pd.get_dummies(wine_df, columns=['country', 'variety'])

In [13]:
wine_df.head()

,Unnamed: 0,points,price,ripe,crisp,bright,dry,full,sweet,fresh,...,variety_Malbec,variety_Merlot,variety_Other,variety_Pinot Noir,variety_Portuguese Red,variety_Red Blend,variety_Riesling,variety_Rose,variety_Sauvignon Blanc,variety_Syrah
0,1,87,15.0,1,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
1,2,87,14.0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,3,87,13.0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,4,87,65.0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,5,87,15.0,0,0,0,0,1,0,1,...,0,0,1,0,0,0,0,0,0,0


In [14]:
# Split our preprocessed data into our features and target arrays
y = wine_df["Points"].values
X = wine_df.drop(["Points"],1).values

C:\Users\jenva\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

In [16]:
lr = LogisticRegression(multi_class='ovr',solver='lbfgs',max_iter=100)
model = lr.fit(X_train, y_train)

accuracy = model.score(X_test, y_test)
print ("Accuracy is {}".format(accuracy))

Accuracy is 0.6204107031736155


In [19]:
import pickle
pickle.dump(lr, open("../model.p", "wb"))

In [21]:
featureList = []

for col in X.columns:
    feature = {
        "Price" : col, 
        "Points" :col
    }
    featureList.append(feature)

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [22]:
pd.DataFrame(featureList).to_csv("../features.csv", index=False)